<a href="https://colab.research.google.com/github/PedroJuiz/SupervisedLearning/blob/main/Incremental_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Descomenta esta linea para instalar los paquetes
!pip install -q scikit-learn==0.24.2

<center>
<img src="mioti_logo.jpeg" width="200px"/>
<p style="font-size: 18px"><b>Machine learning 3</b><br/>Diego García Morate - diegogm@faculty.mioti.es</p>
</center>
<br/>

# Challenge S4: Analizando bosques

# Objetivos

El objetivo de este challenge es ser capaz de clasificar un dataset de tipos de suelo mediante las técnicas de aprendizaje incremental vistas en el worksheet.

Importante: sólo hay una única restricción: ¡**No puedes en ningún momento de este challenge cargar el dataset completo en memoria!** (a excepción que se indique lo contrario).

## Configuración del entorno

In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
import sklearn

import random

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.linear_model import Perceptron

## Covertype dataset

El dataset que vamos a utilizar en este challenge se denomina Covertype dataset (https://archive.ics.uci.edu/ml/datasets/covertype). Este dataset está formado por más de 500.000 observaciones de regiones de 30x30m de bosques en el parque nacional Roosevelt en Colorado.

Las variables de las que está compuesto el dataset según la documentación (`dataset/covtype.info`) son las siguientes:

<pre style="font-size:10px">
Name                                     Data Type    Measurement                       Description

Elevation                               quantitative    meters                       Elevation in meters
Aspect                                  quantitative    azimuth                      Aspect in degrees azimuth
Slope                                   quantitative    degrees                      Slope in degrees
Horizontal_Distance_To_Hydrology        quantitative    meters                       Horz Dist to nearest surface water features
Vertical_Distance_To_Hydrology          quantitative    meters                       Vert Dist to nearest surface water features
Horizontal_Distance_To_Roadways         quantitative    meters                       Horz Dist to nearest roadway
Hillshade_9am                           quantitative    0 to 255 index               Hillshade index at 9am, summer solstice
Hillshade_Noon                          quantitative    0 to 255 index               Hillshade index at noon, summer soltice
Hillshade_3pm                           quantitative    0 to 255 index               Hillshade index at 3pm, summer solstice
Horizontal_Distance_To_Fire_Points      quantitative    meters                       Horz Dist to nearest wildfire ignition points
Wilderness_Area (4 binary columns)      qualitative     0 (absence) or 1 (presence)  Wilderness area designation
Soil_Type (40 binary columns)           qualitative     0 (absence) or 1 (presence)  Soil Type designation
Cover_Type (7 types)                    integer         1 to 7                       Forest Cover Type designation
</pre>

Utilizaremos como variable objetivo la última variable `Cover_Type` que indica la categoría de bosque en la que se clasifica ese área:

<pre style="font-size:10px">
Forest Cover Type Classes:

1 -- Spruce/Fir
2 -- Lodgepole Pine
3 -- Ponderosa Pine
4 -- Cottonwood/Willow
5 -- Aspen
6 -- Douglas-fir
7 -- Krummholz
</pre>

# Análisis previo del dataset


Carga un fragmento del dataset (`dataset/covtype.csv`), verifica que las variables cargadas coinciden con el archivo de información del dataset.

In [ ]:
batch_size = 10000

with open('dataset/covtype.csv', 'r') as file:
    iterator = pd.read_csv(file, chunksize=batch_size, delimiter=";")

    for batch in iterator:
        print (f'Cargado batch de datos de {len(batch)} elementos.')

Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado bat

# Conjunto de entrenamiento y test

Divide los datos en conjunto de entrenamiento y test (70% entrenamiento y 30%). Para ello deberás leer el fichero una sola vez y guardar los resultados en dos ficheros distintos: `train.csv` y `test.csv`.

Recuerda que no puedes cargar los datos en memoria en su totalidad y usar `train_test_split`.

In [ ]:
batch_size = 10000
train_ratio = 0.7

with open('dataset/covtype.csv', 'r') as file:
    iterator = pd.read_csv(file, chunksize=batch_size, delimiter=",")

    with open('train.csv', 'w') as train_file, open('test.csv', 'w') as test_file:
        first_batch = True
        for batch in iterator:
            print(f'Cargado batch de datos de {len(batch)} elementos.')

            train_batch = batch.sample(frac=train_ratio, random_state=42)
            test_batch = batch.drop(train_batch.index)

            if first_batch:
                train_batch.to_csv(train_file, index=False, header=True)
                test_batch.to_csv(test_file, index=False, header=True)
                first_batch = False
            else:
                train_batch.to_csv(train_file, index=False, header=False, mode='a')
                test_batch.to_csv(test_file, index=False, header=False, mode='a')

print("Datos divididos en train.csv y test.csv")

Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado batch de datos de 10000 elementos.
Cargado bat

# ¿Qué deberías hacer si quieres hacer que la división entre entrenamiento y test sea estratificado?

Te recuerdo que una división se denomina estratificada cuando garantiza que la distribución de la variable objetivo se mantiene entre el conjunto de entrenamiento y test.

No hace falta que implementes la función, con explicar como lo resolverías es suficiente.

Para obtener una división de entrenamiento y test estratificado, mientras se lee el csv por lotes, se debería calcular la distribucion de la variable objetivo en el dataset completo sin cargar los datos en memoria en su totalidad. Esto podría hacerse leyendo linea por linea contando la frecuencia de cada clase.

# Entrenamiento incremental

Entrena de forma incremental un `Perceptron` para este conjunto de datos. El dataset de test lo puedes cargar en memoria en su totalidad. Te recomiendo que utilices funciones al menos con el preprocesamiento para poder reutilizarlo.

Intenta obtener los mejores resultados posibles. Algunas de las técnicas explicadas en el worksheet te ayudarán a obtener mejores resultados.

Por último indica cual sería el porcentaje de acierto del modelo. ¿es único? ¿cambia a lo largo del tiempo?.

In [ ]:
def preprocesar_datos(X, scaler=None):
    if scaler is None:
        scaler = StandardScaler()
        X_procesada = scaler.fit_transform(X)
    else:
        X_procesada = scaler.transform(X)
    return X_procesada, scaler

In [ ]:
test_data = pd.read_csv('test.csv', delimiter=",")
X_test = test_data.drop('Cover_Type', axis=1)
y_test = test_data['Cover_Type']

X_test, scaler = preprocesar_datos(X_test)

perceptron = Perceptron()

with open('train.csv', 'r') as file:
    iterator = pd.read_csv(file, chunksize=batch_size, delimiter=",")

    for batch in iterator:
        X_train = batch.drop('Cover_Type', axis=1)
        y_train = batch['Cover_Type']

        X_train, _ = preprocesar_datos(X_train, scaler)

        perceptron.partial_fit(X_train, y_train, classes=np.unique(y_test))

y_pred = perceptron.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy : {accuracy}")

Accuracy : 0.4148154947677621


# Entrenamiento no incremental

Carga el dataset completo en memoria, dividelo en conjunto de entrenamiento y test y entrenalo en un único paso. Calcula el porcentaje de acierto.

In [ ]:
df = pd.read_csv('dataset/covtype.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581012 non-null  int64
 1   Aspect                              581012 non-null  int64
 2   Slope                               581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64
 6   Hillshade_9am                       581012 non-null  int64
 7   Hillshade_Noon                      581012 non-null  int64
 8   Hillshade_3pm                       581012 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64
 10  Wilderness_Area1                    581012 non-null  int64
 11  Wilderness_Area2                    581012 non-null 

In [ ]:
X = df.drop('Cover_Type', axis=1)
y = df['Cover_Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

perceptron = Perceptron()
perceptron.fit(X_train, y_train)

Perceptron()

In [ ]:
y_pred = perceptron.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Porcentaje de acierto: {accuracy}")

Porcentaje de acierto: 0.6105539746649532


# Conclusiones

Compara los resultados obtenidos del aprendizaje incremental con el no incremental. Explica las ventajas e inconvenientes de cada uno de los enfoques.

Indica también algún punto de mejora que se ocurra.

En este caso, se obtienen mejores resultados cargando el dataset entero en memoria que realizando un entrenamiento incremental. Esto puede deberse a que en el conjunto de train y test del modelo de aprendizaje incremental esten las clases desbalanceadas, debido a que no hemos tenido en cuenta lo comentado previamente en el notebook acerca de división estratificada.
A continuacion, se exponen una serie de ventajas e inconvenientes para cada método:

#### APRENDIZAJE INCREMENTAL

##### Ventajas
- Capacidad para manejar grandes conjuntos de datos que no pueden cargarse completamente en la memoria.
- Permite actualizar el modelo en tiempo real a medida que llegan nuevos datos sin necesidad de volver a entrenar con todo el conjunto de datos acumulado.
- Puede ser más rápido en ciertos casos, ya que el modelo se actualiza con cada lote de datos en lugar de esperar a que se procesen todos los datos.

##### Desventajas
- La convergencia del modelo puede ser más lenta o menos estable, ya que los pesos se actualizan con cada lote de datos en lugar de minimizar el error en todo el conjunto de datos.
- Puede ser más difícil de implementar y gestionar, ya que se deben manejar los lotes de datos y el proceso de actualización del modelo.

#### APRENDIZAJE NO INCREMENTAL

##### Ventajas
- Suele ser más fácil de implementar y gestionar, ya que el modelo se entrena en un único paso utilizando todo el conjunto de datos.
- Puede tener una convergencia más rápida y estable, ya que el modelo se optimiza utilizando todo el conjunto de datos en lugar de actualizarse con cada lote de datos.

##### Desventajas
- No es adecuado para conjuntos de datos muy grandes que no pueden cargarse completamente en la memoria.
- No permite actualizar el modelo en tiempo real a medida que llegan nuevos datos; es necesario volver a entrenar el modelo con todo el conjunto de datos acumulado.
- Puede ser más lento en ciertos casos, ya que el modelo debe procesar todo el conjunto de datos antes de que se realicen las actualizaciones de los pesos.